In [1]:
# import the needed libraries

import pandas as pd 
import numpy as np
import optuna

from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from lazypredict.Supervised import LazyClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#  load the dataset

loan_data = pd.read_csv(r'C:\Users\hp\Desktop\machine-learning-notes\data\loan_data.csv')

In [3]:
# inspect the dataset

loan_data.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.00,female,Master,71948.00,0,RENT,35000.00,PERSONAL,16.02,0.49,3.00,561,No,1
1,21.00,female,High School,12282.00,0,OWN,1000.00,EDUCATION,11.14,0.08,2.00,504,Yes,0
2,25.00,female,High School,12438.00,3,MORTGAGE,5500.00,MEDICAL,12.87,0.44,3.00,635,No,1
3,23.00,female,Bachelor,79753.00,0,RENT,35000.00,MEDICAL,15.23,0.44,2.00,675,No,1
4,24.00,male,Master,66135.00,1,RENT,35000.00,MEDICAL,14.27,0.53,4.00,586,No,1


In [4]:
# split the data into X and y

X = loan_data.drop(columns=['loan_status']).copy()
y = loan_data['loan_status'].values

In [5]:
# create a list of categorical colunms

cat_columns = ['person_gender', 'person_education',
       'person_home_ownership', 'loan_intent','previous_loan_defaults_on_file']
cont_columns = [x for x in X.columns if x not in cat_columns]

In [6]:
# encode the categorical variables

encoder = LabelEncoder()
for col in cat_columns:
    X[col] = encoder.fit_transform(X[col])

In [7]:
# extract the validation set from the dataframe

X, X_val, y, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

In [8]:
# scale the numric variables.
scaler = StandardScaler()
columns = list(X.columns)
X = scaler.fit_transform(X)
X = pd.DataFrame(data = X, columns=columns)
X.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file
0,-0.79,-1.11,1.53,0.24,-0.73,-1.17,-0.73,0.28,0.01,-1.03,-0.74,-0.29,0.99
1,1.19,-1.11,-1.18,-0.19,0.75,0.91,-0.67,1.43,1.01,-0.68,2.35,-1.22,-1.01
2,-0.30,-1.11,1.53,-0.39,-0.89,0.91,0.27,-1.46,0.76,0.92,-0.23,-0.29,0.99
3,-0.30,-1.11,-1.18,-0.28,-0.89,0.91,-0.57,0.28,1.21,-0.45,-0.74,0.60,-1.01
4,0.36,0.90,-0.50,-0.15,0.26,0.91,0.38,-1.46,-1.87,0.35,0.29,1.34,-1.01


#### TRAIN THE BASEMODEL

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, 
                                                    random_state=23, stratify=y)
base_classifier = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,_ = base_classifier.fit(X_train, X_test, y_train, y_test)
models


 38%|███▊      | 11/29 [00:15<00:38,  2.12s/it]

In [ ]:
# create objective function
from sklearn.ensemble import RandomForestClassifier

def objective(trial: int) -> float:
  
  # specify the params
  n_estimators = trial.suggest_int('n_estimators', 5, 200)
  max_depth = trial.suggest_int('max_depth', 2, 20)
  criterion = trial.suggest_categorical('criterion', ['entropy','log_loss','gini'])
  
  classifier = RandomForestClassifier(n_estimators = n_estimators,
                                      max_depth = max_depth,
                                      criterion  )